In [1]:
from pathlib import Path
import itertools
from matplotlib import pyplot as plt

stim_data_dir = Path("..", "data", "Nimstim faces")
grayscale = True

open_files = [x for x in stim_data_dir.glob("*_O.bmp")]
closed_files = [x for x in stim_data_dir.glob("*_C.bmp")]

#     img = open_files[0]
data_file = list(itertools.islice(open_files, 1))[0]


In [2]:
from protosc.preprocessing import GreyScale, ViolaJones, CutCircle
from protosc.pipe_complex import PipeComplex
from protosc.feature_extraction import FourierFeatures
from protosc.io import ReadImage

In [14]:
pipe1 = ReadImage()*(ViolaJones()+ViolaJones(10))*CutCircle()*FourierFeatures()
pipe2 = ReadImage()*GreyScale()*CutCircle()*FourierFeatures()

pipe_complex = pipe1 + pipe2
print(pipe_complex)

Pipe: ReadImage -> ViolaJones_20 -> CutCircle -> FourierFeatures_a8s7
Pipe: ReadImage -> ViolaJones_10 -> CutCircle -> FourierFeatures_a8s7
Pipe: ReadImage -> GreyScale -> CutCircle -> FourierFeatures_a8s7


In [11]:
images = pipe_complex.execute(open_files, max_depth=1)

(200, 200, 3)
(200, 200, 3)
(650, 506, 1)
(200, 200, 3)
(200, 200, 3)
(624, 506, 1)
(200, 200, 3)
(200, 200, 3)
(620, 506, 1)
(200, 200, 3)
(200, 200, 3)
(650, 506, 1)
(200, 200, 3)
(200, 200, 3)
(625, 506, 1)
(200, 200, 3)
(200, 200, 3)
(650, 506, 1)
(200, 200, 3)
(200, 200, 3)
(614, 506, 1)
(200, 200, 3)
(200, 200, 3)
(650, 506, 1)
(200, 200, 3)
(200, 200, 3)
(629, 506, 1)
(200, 200, 3)
(200, 200, 3)
(620, 506, 1)
(200, 200, 3)
(200, 200, 3)
(650, 506, 1)
(200, 200, 3)
(200, 200, 3)
(650, 506, 1)
(200, 200, 3)
(200, 200, 3)
(650, 506, 1)
(200, 200, 3)
(200, 200, 3)
(650, 506, 1)
(200, 200, 3)
(200, 200, 3)
(623, 506, 1)
(200, 200, 3)
(200, 200, 3)
(650, 506, 1)
(200, 200, 3)
(200, 200, 3)
(650, 506, 1)
(200, 200, 3)
(200, 200, 3)
(650, 506, 1)
(200, 200, 3)
(200, 200, 3)
(606, 506, 1)
(200, 200, 3)
(200, 200, 3)
(650, 506, 1)
(200, 200, 3)
(200, 200, 3)
(625, 506, 1)
(200, 200, 3)
(200, 200, 3)
(650, 506, 1)
(200, 200, 3)
(200, 200, 3)
(650, 506, 1)
(200, 200, 3)
(200, 200, 3)
(650, 